In [20]:
import networkx as nx

import matplotlib.pyplot as plt

from tqdm import tqdm
import re
from sklearn.model_selection import train_test_split

In [21]:
import pandas as pd
import csv
import numpy as np
import random
# import wikipedia sentences
import os
pd.set_option('display.max_colwidth', 200)
%matplotlib inline
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))


Files in '/Users/wz/Desktop': ['KEPLER.pdf', 'paper', 'Prototype 0.0.ipynb', 'fact_DB_test.csv', 'Analogical Inference for Multi-relational Embeddings.pdf', '.$factual pipeline.drawio.bkp', 'video', '.$factual pipeline.drawio.dtmp', '.DS_Store', '.localized', 'generate_data.ipynb', 'Resources', 'thesis备份.docx', 'fact_DB_train.csv', 'Partial Figure', 'DBpedia_query_train.csv', 'DBpedia_query_test.csv', 'fact_aware language modeling', '.ipynb_checkpoints', 'experts.py', 'linking example.png', 'dbpedia.3.8.tsv', 'mcmc integration project.pdf', 'Fact-aware modeling', 'Relational inductive biases, deep learning, and graph networks.pdf']


In [22]:
a = pd.read_csv("/Users/wz/Desktop/dbpedia.3.8.tsv",sep='\t',names=['s', 'p', 'o'])
a.shape

(11024066, 3)

In [4]:
# # shrink size 
a = a.sample(n=200000, random_state=1)
a.shape

(200000, 3)

In [23]:
# extract subject
source = a.loc[:,"s"]

relations = a.loc[:,"p"]

target = a.loc[:,"o"]

# print(len(source))
# print(len(target))
# print(len(relations))

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})



In [24]:
kg_df

,source,target,edge
0,<db:Animal_Farm>,<db:George_Orwell>,<dbo:author>
1,<db:Nineteen_Eighty-Four>,<db:George_Orwell>,<dbo:author>
2,<db:Homage_to_Catalonia>,<db:George_Orwell>,<dbo:author>
3,<db:Keep_the_Aspidistra_Flying>,<db:George_Orwell>,<dbo:author>
4,<db:Down_and_Out_in_Paris_and_London>,<db:George_Orwell>,<dbo:author>
...,...,...,...
11024061,<db:Mike_Cochraine>,<db:MidAmerica_Nazarene_University>,<dbo:coachedTeam>
11024062,<db:Gordon_DeGraffenreid>,<db:MidAmerica_Nazarene_University>,<dbo:coachedTeam>
11024063,<db:Carboxypeptidase_A6__Carboxypeptidase_A6__1>,<db:Carboxypeptidase_A6__Carboxypeptidase_A6__1__2>,<dbo:orthologousGene>
11024064,<db:KIAA1704>,<db:KIAA1704__2>,<dbo:orthologousGene>


In [25]:
# kg_df[kg_df.source.str.contains('Joe_Biden',case=False)]

In [26]:
# kg_df[kg_df.source.str.contains('Donald_Trump',case=False)&kg_df.target.str.contains('United_States',case=False)]

In [27]:
# kg_df[kg_df.edge.str.contains('president',case=False)]

In [28]:
# kg_df[kg_df.source.str.contains('Donald_Trump',case=False)]

# search_values = ['Donald_Trump','Joe_Biden']
# kg_df[kg_df.source.str.contains('|'.join(search_values ))]

In [ ]:
# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [ ]:
# plt.figure(figsize=(12,12))

# pos = nx.spring_layout(G)
# nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
# plt.show()

In [ ]:
# H = G.subgraph(['<db:Anton_LaVey>','<dbo:author>'])

In [ ]:
# nx.descendants(G, '''<db:Sharpe's_Revenge_(novel)>''')

In [ ]:
# 1
# spl = dict(nx.all_pairs_shortest_path_length(G))

#spl['''<db:Sharpe's_Revenge_(novel)>''']["<db:Sharpe's_Christmas>"]

In [ ]:
# 2
# length=nx.single_source_shortest_path_length(G,'''<db:Sharpe's_Revenge_(novel)>''')
# max(length.values())

In [ ]:
# list(H)

In [ ]:
# list(H.edges)

In [ ]:
# G.nodes

In [ ]:
# list(G)

### form dataset

In [7]:
# remove useless lines
DBpedia = kg_df.copy()

# remove the <db: xxx > sign 
DBpedia['source'] = kg_df['source'].apply(lambda x: x[4:-1]) 
DBpedia['edge'] = kg_df['edge'].apply(lambda x: x[5:-1])
DBpedia['target'] = kg_df['target'].apply(lambda x: x[4:-1])
# remove the "_" in between
DBpedia['source'] = DBpedia['source'].apply(lambda x: re.sub('_', ' ', x))
DBpedia['edge'] = DBpedia['edge'].apply(lambda x: re.sub('_', ' ', x))
DBpedia['target'] = DBpedia['target'].apply(lambda x: re.sub('_', ' ', x))

# cleaning
#print(DBpedia[DBpedia.source.str.contains(r'[@#&$%/*]')])

# select and then merge rows
# with special characters
print(DBpedia[DBpedia.source.str.contains(r'[@#&$%/*]') |
         DBpedia.target.str.contains(r'[@#&$%/*]') |
         DBpedia.edge.str.contains(r'[@#&$%/*]') ])
 
# drop the rows
print(DBpedia.drop(DBpedia[DBpedia.source.str.contains(r'[@#&$%/*]') |
         DBpedia.target.str.contains(r'[@#&$%/*]') |
         DBpedia.edge.str.contains(r'[@#&$%/*]') ].index))
DBpedia = DBpedia.drop(DBpedia[DBpedia.source.str.contains(r'[@#&$%/*]') |
         DBpedia.target.str.contains(r'[@#&$%/*]') |
         DBpedia.edge.str.contains(r'[@#&$%/*]') ].index)

# new G
# create a directed-graph from a dataframe
DB_G=nx.from_pandas_edgelist(DBpedia, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())



                                                      source  \
8452678          FK Borac Banja Luka  Boris Raspudi%C4%87  1   
8729051                      CF Balaguer  Adri%C3%A0 Pons  1   
4086340                                Extra (service areas)   
3971619                                         Paul O'Brien   
3936755                            Colorado State University   
...                                                      ...   
1857502                           Jos%C3%A9 Luis Garc%C3%ADa   
6403491                                             Seldiren   
2116574                     %C4%8Ceslovas Jur%C5%A1%C4%97nas   
10461171  Genting Sempah %E2%80%93 Genting Highlands Highway   
1704883                                  Petar Gra%C4%8Danin   

                                target                            edge  
8452678            FK Borac Banja Luka                            team  
8729051                    CF Balaguer                            team  
4086340    p

In [8]:
# shrink size 
DBpedia = DBpedia.sample(n=100000, random_state=1)
# query version DB
DBpedia_new = DBpedia.copy()
# form query and response
DBpedia_new['query_response'] = DBpedia['source'] + " " + DBpedia['edge']+ " " + DBpedia['target']
DBpedia_new['fact'] = 1

DBpedia_new = DBpedia_new[['query_response', 'fact']].copy()
DBpedia_new

,query_response,fact
1709708,"John Harper (politician) birthPlace Park Ridge, Illinois",1
7253050,Mercy (Steve Jones album) recordLabel MCA Inc.,1
10932931,Ensign Records distributingCompany Chrysalis Records,1
3168104,John Benedetto knownFor Wavelet,1
5845748,Shirvanshah Ali II predecessor Shirvanshah Manuchehr I,1
...,...,...
3703804,"Newport High School, Bettws Lane type Comprehensive school",1
5496563,Purple Vista (Omni Series 2) genre Rock music,1
5799175,Lucidity (video game) computingPlatform Xbox 360,1
6545793,"Greece (CDP), New York timeZone Eastern Time Zone (North America)",1


In [9]:
DBpedia

,source,target,edge
1709708,John Harper (politician),"Park Ridge, Illinois",birthPlace
7253050,Mercy (Steve Jones album),MCA Inc.,recordLabel
10932931,Ensign Records,Chrysalis Records,distributingCompany
3168104,John Benedetto,Wavelet,knownFor
5845748,Shirvanshah Ali II,Shirvanshah Manuchehr I,predecessor
...,...,...,...
3703804,"Newport High School, Bettws Lane",Comprehensive school,type
5496563,Purple Vista (Omni Series 2),Rock music,genre
5799175,Lucidity (video game),Xbox 360,computingPlatform
6545793,"Greece (CDP), New York",Eastern Time Zone (North America),timeZone


In [10]:
DBpedia_new

,query_response,fact
1709708,"John Harper (politician) birthPlace Park Ridge, Illinois",1
7253050,Mercy (Steve Jones album) recordLabel MCA Inc.,1
10932931,Ensign Records distributingCompany Chrysalis Records,1
3168104,John Benedetto knownFor Wavelet,1
5845748,Shirvanshah Ali II predecessor Shirvanshah Manuchehr I,1
...,...,...
3703804,"Newport High School, Bettws Lane type Comprehensive school",1
5496563,Purple Vista (Omni Series 2) genre Rock music,1
5799175,Lucidity (video game) computingPlatform Xbox 360,1
6545793,"Greece (CDP), New York timeZone Eastern Time Zone (North America)",1


In [11]:
# save the query & response dataset
DBpedia_query = DBpedia.copy()
DBpedia_query['query'] = DBpedia['source'] + " " + DBpedia['edge']
DBpedia_query['response'] =  DBpedia['target']
DBpedia_query = DBpedia_query[['query', 'response']].copy()
# save
DBpedia_query_train, DBpedia_query_test = train_test_split(DBpedia_query, test_size=0.2)
DBpedia_query_train.to_csv('DBpedia_query_train.csv', encoding='utf-8')
DBpedia_query_test.to_csv('DBpedia_query_test.csv', encoding='utf-8')

In [12]:


postive, negative = train_test_split(DBpedia_new, test_size=0.4)

In [13]:
negative


,query_response,fact
8255870,Alan Gilzean team Scottish Football League XI,1
5635745,Themes on an Occult Theory genre Grindcore,1
5326370,Zombie Apocalypse (video game) developer Nihilistic Software,1
3439704,Adrian Smith (strongman) occupation Adrian Smith (strongman) 1,1
3681547,"Saint-Nazaire, Quebec type Types of municipalities in Quebec",1
...,...,...
7960101,Marine Wing Support Squadron 271 militaryBranch United States Marine Corps,1
1383743,Clem Lewis birthPlace Wales,1
9815433,HERAS-AF operatingSystem Cross-platform,1
8353108,Mark Bailey (footballer) team Peterborough United F.C.,1


In [14]:
random_index = random.choice(np.array(negative.index))
kg_df.loc[random_index ,"target"]

'<db:Macedonia_national_football_team>'

In [15]:
# negative sampling
for index, row in tqdm(negative.iterrows()):
    
    # print(row['query_response'], row['fact'])
    
    # inialize
    random_index = random.choice(np.array(negative.index))
    
    # fetch a false response
    false_response = DBpedia.loc[random_index ,"target"]
    
    # print(false_response)
    
    # check it is not connected
    root = DBpedia.loc[index ,"target"] # original target
    
    # print(root)
    while false_response in nx.descendants(DB_G, root):
        random_index = random.choice(np.array(negative.index))
        # fetch another a false response
        false_response = DBpedia.loc[random_index ,"target"]
    # update this row
    negative.loc[index, 'query_response'] = DBpedia.loc[index,'source'] + " " + DBpedia.loc[index,'edge']+ " " + false_response
    negative.loc[index, 'fact'] = 0

40000it [00:05, 7016.73it/s]


In [16]:
vertical_concat = pd.concat([postive, negative], axis=0)
# shuffle the rows
query_DB_fact = vertical_concat.sample(frac=1)
query_DB_fact

,query_response,fact
5031131,Tricarpelema pumilum family RCD Espanyol,0
1907740,Vitaly Sevastyanov deathPlace Russia,1
3789282,"Princess Louise Margaret of Prussia restingPlace Windsor, Berkshire",1
6569077,"Wood Buffalo, Alberta timeZone Mountain Time Zone",1
10362034,WWTN programmeFormat Talk radio,1
...,...,...
1442575,Christos Efthymiou birthPlace Cyprus,1
9800956,Eastcliff (mansion) architect Australia,0
8650503,Marco Giampaolo team A.S.D. Licata 1931,1
3501722,Tufunde Cive type Basidiomycota,0


In [17]:
# check sum
print(query_DB_fact.loc[:,'fact'].sum())
print(DBpedia_new.loc[:,'fact'].sum())

60000
100000


In [18]:
fact_DB_train, fact_DB_test = train_test_split(query_DB_fact, test_size=0.2)
fact_DB_train.to_csv('fact_DB_train.csv', encoding='utf-8')
fact_DB_test.to_csv('fact_DB_test.csv', encoding='utf-8')